In [1]:
# Imports
import copy
import pandas as pd
import pickle
import random
import rlcard
import rlcard.envs
import torch
import yaml

from datetime import datetime
from rlcard.agents import RandomAgent
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM

DEPRECATION: Loading egg at /mnt/appl/software/Python/3.11.5-GCCcore-13.2.0/lib/python3.11/site-packages/flann-1.9.2-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
# Helper functions
def extract_choice(text):
    text = to_lower(text)
    last_hit_index = text.rfind("hit")
    last_stand_index = text.rfind("stand")
    if last_hit_index > last_stand_index:
        return "hit"
    elif last_stand_index > last_hit_index:
        return "stand"
    else:
        return None

def to_lower(str):
    lowercase_string = str.lower()
    return lowercase_string

def card2string(cardList):
    str = ''
    str = ','.join(cardList)
    str = str.replace('C', 'Club ')
    str = str.replace('S', 'Spade ')
    str = str.replace('H', 'Heart ')
    str = str.replace('D', 'Diamond ')
    str = str.replace('T', '10')
    return str

def blackjack_value(hand):
    total = 0
    ace_count = 0
    
    for card in hand:
        rank = card[1:]  
        
        if rank == 'A':
            total += 11
            ace_count += 1
        elif rank in {'J', 'Q', 'K', 'T', '10'}:
            total += 10
        elif rank.isdigit() and 2 <= int(rank) <= 9:
            total += int(rank)
    
    while total > 21 and ace_count > 0:
        total -= 10  
        ace_count -= 1
    
    return total

def color_strategy(val):
    """Return a CSS background-color depending on the action."""
    if val == "S":
        return "background-color: gold"      # Stand color
    elif val == "H":
        return "background-color: white"     # Hit color
    else:
        return ""  # No styling by default

In [3]:
# DeepSeek R1 model
class DEEPSEEKR1:
    def __init__(self) -> None:
        with open("../config.yaml", "r") as file:
            config = yaml.safe_load(file)
            api_key = config["Keys"]["DEEPSEEKR1PAID"]

        self.model = config["IDs"]["DEEPSEEKR1PAID"]

        self.client = OpenAI(
            base_url=config["Providers"]["DEEPSEEK"],
            api_key=api_key,
        )

    def response(self, mes):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=mes)

        return response.choices[0].message.content

In [3]:
# DeepSeek R1 Qwen model
class DEEPSEEKR1QWEN:
    def __init__(self) -> None:
        self.tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", trust_remote_code=True)    
        self.model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", 
                                                          device_map="auto", 
                                                          torch_dtype="auto",
                                                          attn_implementation="flash_attention_2",
                                                          trust_remote_code=True)

        print(self.model.hf_device_map)

    def response(self, mes):
        input_ids = self.tokenizer.apply_chat_template(
            mes,
            add_generation_prompt=True,  
            return_tensors="pt"
        ).to(self.model.device)

        attention_mask = input_ids.ne(self.tokenizer.pad_token_id).long()

        outputs = self.model.generate(
            input_ids,
            max_new_tokens=10000,  
            temperature=0.6,
            top_p = 0.95,
            attention_mask=attention_mask,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id 
        )

        generated_ids = outputs[:, input_ids.shape[1]:]
        response = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        return response
        # return re.split(r'</think>\s*', response, maxsplit=1)[1]

In [4]:
# Blackjack Agent
game_style = 'agentpro'
now = datetime.now()
formatted = now.strftime("%Y-%m-%d %H:%M:%S")
storage_name = '../Agent-Pro/my_data/DeepSeek R1/' + formatted
response = ''

class LlmAgent(RandomAgent):
    def __init__(self, num_actions):
        super().__init__(num_actions)
        self.llm = DEEPSEEKR1QWEN()

    # @staticmethod
    def step(self, state):
        deal_card = state['raw_obs']['dealer hand']
        hand_card = state['raw_obs']['player0 hand']
        p = []
        begin_info = "You are a player in blackjack. Please beat the dealer and win the game.\n"
        game_rule = "Game Rule:\n1. Please try to get your card total to as close to 21 as possible, without going over, and still having a higher total than the dealer.\n2. If anyone's point total exceeds 21, he or she loses the game. \n3. You can only choose one of the following two actions: {\"Stand\", \"Hit\"}. If you choose to Stand, you will stop taking cards and wait for the dealer to finish. If you choose to Hit, you can continue to take a card, but there is also the risk of losing the game over 21 points. \n4. After all players have completed their hands, the dealer reveals their hidden card. Dealers must hit until their cards total 17 or higher.\n"
        game_info = "The dealer's current card is {" + card2string(deal_card) + "}. The dealer has another hidden card. You don't know what it is. Your current cards are {" + card2string(hand_card) + "}. "

        if game_style == 'Vanilla':
            p.append({"role": "system", "content": begin_info + game_rule})
            game_info += "Please output your action in following format: ###My action is {your action}, without any other text."
            p.append({"role": "user", "content": game_info})

        if game_style == 'Radical':
            begin_info = "You are an aggressive player of blackjack who likes to take risks to earn high returns. Please beat the dealer and win the game."
            p.append({"role": "system", "content": begin_info + game_rule})
            game_info += "Please output your action in following format: ###My action is {your action}, without any other text."
            p.append({"role": "user", "content": game_info})

        if game_style == 'ReAct':
            p.append({"role": "system", "content": begin_info + game_rule})
            game_info += "Please first think and reason about the current hand and then generate your action as follows: ###My thought is {Your Thought}. My action is {your action}."
            p.append({"role": "user", "content": game_info})

        if game_style == 'ReFlexion':
            p.append({"role": "system", "content": begin_info + game_rule})
            game_info += "Please first think and reason about the current hand and then generate your action as follows: ###My thought is {Your Thought}. My action is {your action}."
            p.append({"role": "user", "content": game_info})
            llm_res = llm.response(p)
            p.append({"role": "assistant", "content": llm_res})
            reflexion_info = "Please carefully check the response you just output, and then refine your answer . The final output is also in following format: ###My thought is {Your Thought}. My action is {your action}."
            p.append({"role": "user", "content": reflexion_info})
            
        if game_style == 'agentpro':
            begin_info = "I will describe the situation. You have to reason through this in 3-5 steps, then stop. The description begins now. You are an aggressive player of blackjack who likes to take risks to earn high returns. Please beat the dealer and win the game."
            # p.append({"role": "user", "content": begin_info + game_rule})
            game_info += "Please read the behavoiral guideline and world modeling carefully. Then you should analyze your own cards and your strategies in Self-belief and then analyze the dealer cards in World-belief. Lastly, please select your action from {\"Stand\",\"Hit\"}.### Output Format: Self-Belief is {Belief about youself}. World-Belief is {Belief about the dealer}. My action is {Your action}. Please output in the given format. Do not write anything else."
            # p.append({"role": "user", "content": game_info})
            p.append({"role": "user", "content": begin_info + game_rule + game_info})
        llm_res = self.llm.response(p)
        p.append({"role": "assistant", "content": llm_res})
        filename = storage_name + '.yaml'
        with open(filename, "a") as yaml_file:
            yaml.dump(p, yaml_file, default_flow_style=False, allow_unicode=True)
        choice = -1
        if extract_choice(llm_res) == "hit":
            choice = 0
        elif extract_choice(llm_res) == "stand":
            choice = 1
        else:
            choice = -1
        return choice

In [5]:
# Results table
results = {17: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           16: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           15: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           14: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           13: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           12: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           11: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
           10: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
            9: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []},
            8: {2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 'A': []}}

In [91]:
# Dictionary to store found seeds
found_seeds = dict()

In [32]:
# Random agent to make env run
agent = RandomAgent(num_actions=2)

In [93]:
# Searching for seeds to launch evaluation
for i in range(100000):
    env = rlcard.make("blackjack", config={"seed": i})
    env.set_agents([agent])
    trajectories, payoffs = env.run(is_training=False)

    for j, situation in enumerate(trajectories[0][:1]):
        last_hand_value = int(trajectories[0][j]['obs'][0])
        if last_hand_value <= 17 and last_hand_value >= 8:
            dealer_value = int(trajectories[0][j]['obs'][1]) 
            if (last_hand_value, dealer_value) not in found_seeds.keys():
                found_seeds[(last_hand_value, dealer_value)] = [i]
            elif len(found_seeds[(last_hand_value, dealer_value)]) < 10:
                found_seeds[(last_hand_value, dealer_value)].append(i)

In [6]:
# DeepSeek agent
llm_agent = LlmAgent(num_actions=2)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/64.0k [00:00<?, ?B/s]

model-00001-of-000008.safetensors:   0%|          | 0.00/8.79G [00:00<?, ?B/s]

...

model-00007-of-000008.safetensors:   0%|          | 0.00/8.78G [00:00<?, ?B/s]

model-00008-of-000008.safetensors:   0%|          | 0.00/4.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 1, 'model.layers.7': 1, 'model.layers.8': 1, 'model.layers.9': 1, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 2, 'model.layers.16': 2, 'model.layers.17': 2, 'model.layers.18': 2, 'model.layers.19': 2, 'model.layers.20': 2, 'model.layers.21': 2, 'model.layers.22': 2, 'model.layers.23': 2, 'model.layers.24': 3, 'model.layers.25': 3, 'model.layers.26': 3, 'model.layers.27': 3, 'model.layers.28': 3, 'model.layers.29': 3, 'model.layers.30': 3, 'model.layers.31': 3, 'model.layers.32': 3, 'model.layers.33': 4, 'model.layers.34': 4, 'model.layers.35': 4, 'model.layers.36': 4, 'model.layers.37': 4, 'model.layers.38': 4, 'model.layers.39': 4, 'model.layers.40': 4, 'model.layers.41': 4, 'model.layers.42': 5, 'model.layers.43': 5, 'model.layers.44

In [8]:
def play_game(env):
    trajectories, payoffs = env.run(is_training=False)
    if len(trajectories[0]) != 0:
        final_state = []
        action_record = []
        state = []
        _action_list = []

        for i in range(1):
            final_state.append(trajectories[i][-1])
            state.append(final_state[i]['raw_obs'])

        action_record.append(final_state[i]['action_record'])
        for i in range(1, len(action_record) + 1):
            _action_list.insert(0, action_record[-i])

    last_hand_value = 0
    for i, situation in enumerate(trajectories[0][:-1]):
        if (i % 2 == 0): # State
            last_hand_value = int(trajectories[0][i]['obs'][0])
            dealer_value = int(trajectories[0][i]['obs'][1]) 
            dealer_value = dealer_value if dealer_value <= 10 else 'A'
        else: # Action
            action = int(situation)
            if last_hand_value >= 8 and last_hand_value <= 17:
                results[last_hand_value][dealer_value].append(action)

    res_str = ('dealer {}, '.format(state[0]['state'][1]) +
                'player {}, '.format(state[0]['state'][0]))
    if payoffs[0] == 1:
        final_res = "win."
    elif payoffs[0] == 0:
        final_res = "draw."
    elif payoffs[0] == -1:
        final_res = "lose."
    p = [{"final cards": res_str, "final results": final_res}]
    filename = storage_name + '.yaml'
    with open(filename, "a") as yaml_file:
        yaml.dump(p, yaml_file, default_flow_style=False, allow_unicode=True)
    return env.get_payoffs()

In [9]:
# Random games
for i in range(1, 1001):
    now = datetime.now()
    formatted = now.strftime("%Y-%m-%d %H:%M:%S")
    storage_name = '../Agent-Pro/my_data/WSB/' + formatted
    env = rlcard.make(
    'blackjack',
    config={
        'game_num_players': 1,
        "seed": random.randint(0, 10**10)
    })
    env.set_agents([llm_agent])
    play_game(env)
    print(f"Current game: {i}")
    if i % 100 == 0:
        with open(f"../dicts/data_{formatted}.pkl", "wb") as f:
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

Current game: 1
Current game: 2
Current game: 3


KeyboardInterrupt: 

In [106]:
# Convert lists into letters
results_copy = copy.deepcopy(results)
for row in results.items():
    for key in row[1].keys():
        row[1][key] = 'H' if round(sum(row[1][key]) / len(row[1][key])) == 0 else 'S'

In [107]:
# Visualize results
df = pd.DataFrame.from_dict(results, orient='index')
styled_df = df.style.map(color_strategy)
styled_df

,2,3,4,5,6,7,8,9,10,A
17,H,S,S,S,S,S,H,H,H,H
16,H,S,H,S,H,H,H,H,H,H
15,H,H,H,H,H,H,H,H,H,H
14,H,H,H,H,H,H,H,H,H,H
13,H,H,H,H,H,H,H,H,H,H
12,H,H,H,H,H,H,H,H,H,H
11,H,H,H,H,H,H,H,H,H,H
10,H,H,H,H,H,H,H,H,H,H
9,H,H,H,H,H,H,H,H,H,H
8,H,H,H,H,H,H,H,H,H,H


In [ ]:
# Report winrate

import os
def calculate_winrate(directory):
    won = 0
    total_games = 0

    for filename in os.listdir(directory):
        if filename.endswith(".yaml") or filename.endswith(".yml"):
            filepath = os.path.join(directory, filename)

            with open(filepath, "r") as file:
                data = yaml.safe_load(file)

                if "final results" in data[-1]:
                    total_games += 1
                    if str(data[-1]["final results"]).strip().lower() == "win.":
                        won += 1

    winrate = (won / total_games * 100) if total_games > 0 else 0
    return won, total_games, winrate

directory_path = "../Agent-Pro/my_data/WSB"
wins, total, winrate = calculate_winrate(directory_path)

print(f"Total Wins: {wins}")
print(f"Total Games: {total}")
print(f"Win Rate: {winrate:.2f}%")

Total Wins: 41
Total Games: 109
Win Rate: 37.61%


# Optimal strategy

<div>
    <img src="../optimal_strategy.png" width="500"/>
</div>